## Выбор начальных условий

Я взял один и тот же датасет для классификации лекарственных средств, где можно поставить обе задачи

Датасет содержит:
- Возраст
- Пол
- Артериальное давление
- Холестерин
- Соотношение натрия к калию
- Лекарственный препарат (название)

Задача классификации заключается в предсказании класса лекарства **`(Drug)`** на основе набора признаков (`Age`, `Sex`, `BP`, `Cholesterol`, `Na_to_K`). Здесь целевая переменная **`Drug`** категориальная, что делает задачу классификацией

*Почему актуальна?*

В реальной медицинской практике врачи часто сталкиваются с задачей выбора подходящего лекарства для лечения пациента на основе его физиологических характеристик (как и на удивление в представленном датасете - возраст, пол, давление и уровень холестерина)

**Метрики:**
1) `Accuracy`: показывает долю правильно классифицированных объектов, поэтому подходит для сбалансированных данных
2) `F1-Score`: учитывает баланс между точностью и полнотой, что важно при несбалансированных данных

Задача регрессии заключается в предсказании уровня **`Na_to_K`** (соотношение натрия к калию в крови). Это непрерывная переменная, что делает задачу регрессией

*Почему актуальна?*

Уровень `Na_to_K` (соотношение натрия к калию) в крови — важный показатель, который врачи используют для диагностики различных заболеваний, таких как гипертензия, почечные нарушения или сердечно-сосудистые заболевания. Это значение часто зависит от возраста, пола, состояния артериального давления и уровня холестерина

**Метрика:**
1) `Mean Squared Error (MSE)`: среднеквадратичная ошибка, чувствительная к большим отклонениям
2) `Mean Absolute Error (MAE)`: средняя абсолютная ошибка, менее чувствительна к выбросам
3) `R^2 (коэффициент детерминации)`: оценивает долю дисперсии, объясняемую моделью, что позволяет понять, насколько хорошо модель объясняет данные


Загружаем датасет, выводим информацию о нём, потом преобразуем категориальные признаки в числовые

Выбираем признаки и целевые переменные на основе того, какие задачи мы решаем. Разделяем данных на тренировочную и тестовую выборки и нормируем

In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, classification_report,
    mean_squared_error, mean_absolute_error, r2_score
)

data = pd.read_csv('./drug200.csv')

print(data.head())
print(data.info())

label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex']) # 0 - Female, 1 - Male
data['BP'] = label_encoder.fit_transform(data['BP'])   # 0 - HIGH, 1 - LOW, 2 - NORMAL
data['Cholesterol'] = label_encoder.fit_transform(data['Cholesterol']) # 0 - HIGH, 1 - NORMAL
data['Drug'] = label_encoder.fit_transform(data['Drug'])

X_class = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']]
y_class = data['Drug']
X_reg = data[['Age', 'Sex', 'BP', 'Cholesterol']]
y_reg = data['Na_to_K']

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_class_train_scaled = scaler.fit_transform(X_class_train)
X_class_test_scaled = scaler.transform(X_class_test)
X_reg_train_scaled = scaler.fit_transform(X_reg_train)
X_reg_test_scaled = scaler.transform(X_reg_test)

   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  DrugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  DrugY
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB
None


## Лабораторная работа №1 (Проведение исследований с алгоритмом KNN)

**3a. Формулировка гипотез**
1. Использовать разные значения параметра n_neighbors
2. Добавить новые признаки: взаимодействие между Age и Na_to_K
3. Проверить влияние удаления выбросов на качество модели

**3g. Выводы**
1. Подбор гиперпараметров и добавление нового признака улучшили качество моделей
2. Улучшенная модель классификации достигает более высокой точности и F1-Score
3. Улучшенная модель регрессии показывает снижение ошибок MSE и повышение R^2 Score

In [114]:
knn_class = KNeighborsClassifier(n_neighbors=5)
knn_class.fit(X_class_train_scaled, y_class_train)
y_class_pred = knn_class.predict(X_class_test_scaled)

print("Метрики для классификации:")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.2f}")

knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred = knn_reg.predict(X_reg_test_scaled)

print("\nМетрики для регрессии:")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred):.2f}")

# проверка гипотез - добавляем новый признак для классификации и регрессии
data['Age_Na_to_K'] = data['Age'] * data['Na_to_K']
X_class = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Age_Na_to_K']]
X_reg = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Age_Na_to_K']]

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

X_class_train_scaled = scaler.fit_transform(X_class_train)
X_class_test_scaled = scaler.transform(X_class_test)
X_reg_train_scaled = scaler.fit_transform(X_reg_train)
X_reg_test_scaled = scaler.transform(X_reg_test)

# подбор гиперпараметра n_neighbors для классификации
param_grid = {'n_neighbors': range(1, 21)}
grid_search_class = GridSearchCV(KNeighborsClassifier(), param_grid, scoring='f1_weighted', cv=5)
grid_search_class.fit(X_class_train_scaled, y_class_train)
print(f"Лучший n_neighbors для классификации: {grid_search_class.best_params_}")

# подбор гиперпараметра n_neighbors для регрессии
grid_search_reg = GridSearchCV(KNeighborsRegressor(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_reg.fit(X_reg_train_scaled, y_reg_train)
print(f"Лучший n_neighbors для регрессии: {grid_search_reg.best_params_}")

knn_class_best = grid_search_class.best_estimator_
knn_class_best.fit(X_class_train_scaled, y_class_train)
y_class_pred_best = knn_class_best.predict(X_class_test_scaled)

knn_reg_best = grid_search_reg.best_estimator_
knn_reg_best.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_best = knn_reg_best.predict(X_reg_test_scaled)

print("\nМетрики для улучшенной классификации:")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_best):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_best, average='weighted'):.2f}")

print("\nМетрики для улучшенной регрессии:")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_best):.2f}")


Метрики для классификации:
Accuracy: 0.88
F1-Score: 0.87

Метрики для регрессии:
MSE: 60.72
MAE: 6.63
R² Score: -0.21
Лучший n_neighbors для классификации: {'n_neighbors': 1}
Лучший n_neighbors для регрессии: {'n_neighbors': 3}

Метрики для улучшенной классификации:
Accuracy: 0.93
F1-Score: 0.93

Метрики для улучшенной регрессии:
MSE: 17.35
MAE: 3.21
R² Score: 0.65


### Имплементация алгоритма машинного обучения 

In [115]:
from collections import Counter

class CustomKNN:
    def __init__(self, n_neighbors=5, mode='classification'):
        self.n_neighbors = n_neighbors
        self.mode = mode

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X):
        X = np.array(X)
        predictions = [self._predict_single(x) for x in X]
        return np.array(predictions)

    def _predict_single(self, x):
        distances = np.linalg.norm(self.X_train - x, axis=1)
        nearest_indices = distances.argsort()[:self.n_neighbors]
        nearest_labels = self.y_train[nearest_indices]

        if self.mode == 'classification':
            most_common = Counter(nearest_labels).most_common(1)
            return most_common[0][0]
        elif self.mode == 'regression':
            return nearest_labels.mean()
        

knn_class_custom = CustomKNN(n_neighbors=5, mode='classification')
knn_class_custom.fit(X_class_train_scaled, y_class_train)
y_class_pred_custom = knn_class_custom.predict(X_class_test_scaled)

print("Метрики для кастомной классификации:")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_custom):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_custom, average='weighted'):.2f}")

knn_reg_custom = CustomKNN(n_neighbors=5, mode='regression')
knn_reg_custom.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_custom = knn_reg_custom.predict(X_reg_test_scaled)

print("\nМетрики для кастомной регрессии:")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_custom):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_custom):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_custom):.2f}")


Метрики для кастомной классификации:
Accuracy: 0.93
F1-Score: 0.92

Метрики для кастомной регрессии:
MSE: 20.36
MAE: 3.60
R² Score: 0.59


## Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией) 

**3a. Формулирование гипотез**
1. Проверить влияние новых признаков
2. Подобрать гиперпараметры модели с использованием кросс-валидации

In [103]:
from sklearn.linear_model import LogisticRegression, LinearRegression

log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_class_train_scaled, y_class_train)
y_class_pred = log_reg.predict(X_class_test_scaled)

lin_reg = LinearRegression()
lin_reg.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred = lin_reg.predict(X_reg_test_scaled)

print("Метрики для классификации (бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.2f}")

print("\nМетрики для регрессии (бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred):.2f}")


data['Age_Na_to_K'] = data['Age'] * data['Na_to_K']
X_class = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Age_Na_to_K']]
X_reg = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Age_Na_to_K']]

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

X_class_train_scaled = scaler.fit_transform(X_class_train)
X_class_test_scaled = scaler.transform(X_class_test)
X_reg_train_scaled = scaler.fit_transform(X_reg_train)
X_reg_test_scaled = scaler.transform(X_reg_test)

# подбор гиперпараметров для логистической регрессии
param_grid_class = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search_class = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_class, scoring='f1_weighted', cv=5)
grid_search_class.fit(X_class_train_scaled, y_class_train)
print(f"Лучший параметр C для логистической регрессии: {grid_search_class.best_params_}")

# подбор гиперпараметров для линейной регрессии
param_grid_reg = {'fit_intercept': [True, False]}
grid_search_reg = GridSearchCV(LinearRegression(), param_grid_reg, scoring='neg_mean_squared_error', cv=5)
grid_search_reg.fit(X_reg_train_scaled, y_reg_train)
print(f"Лучший параметр fit_intercept для линейной регрессии: {grid_search_reg.best_params_}")

log_reg_best = grid_search_class.best_estimator_
log_reg_best.fit(X_class_train_scaled, y_class_train)
y_class_pred_best = log_reg_best.predict(X_class_test_scaled)

lin_reg_best = grid_search_reg.best_estimator_
lin_reg_best.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_best = lin_reg_best.predict(X_reg_test_scaled)

print("\nМетрики для классификации (улучшенный бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_best):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_best, average='weighted'):.2f}")

print("\nМетрики для регрессии (улучшенный бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_best):.2f}")


Метрики для классификации (бейзлайн):
Accuracy: 0.93
F1-Score: 0.93

Метрики для регрессии (бейзлайн):
MSE: 6.71
MAE: 2.11
R² Score: 0.87
Лучший параметр C для логистической регрессии: {'C': 100}
Лучший параметр fit_intercept для линейной регрессии: {'fit_intercept': True}

Метрики для классификации (улучшенный бейзлайн):
Accuracy: 0.97
F1-Score: 0.98

Метрики для регрессии (улучшенный бейзлайн):
MSE: 6.45
MAE: 2.03
R² Score: 0.80


### Имплементация алгоритма машинного обучения 

In [104]:
class LinearRegressionFromScratch:
    def __init__(self, learning_rate=0.01, epochs=1000, regularization=None, reg_lambda=0.01):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.regularization = regularization
        self.reg_lambda = reg_lambda
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.epochs):
            y_predicted = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            if self.regularization == 'l2':
                dw += (self.reg_lambda / n_samples) * self.weights
            elif self.regularization == 'l1':
                dw += (self.reg_lambda / n_samples) * np.sign(self.weights)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

class LogisticRegressionFromScratch:
    def __init__(self, learning_rate=0.01, epochs=1000, regularization=None, reg_lambda=0.01):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.regularization = regularization
        self.reg_lambda = reg_lambda
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            if self.regularization == 'l2':
                dw += (self.reg_lambda / n_samples) * self.weights
            elif self.regularization == 'l1':
                dw += (self.reg_lambda / n_samples) * np.sign(self.weights)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        return [1 if i > 0.5 else 0 for i in y_predicted]

log_reg_scratch = LogisticRegressionFromScratch(learning_rate=0.01, epochs=2000, regularization='l2', reg_lambda=0.1)
log_reg_scratch.fit(X_class_train_scaled, y_class_train)
y_class_pred_scratch = log_reg_scratch.predict(X_class_test_scaled)

lin_reg_scratch = LinearRegressionFromScratch(learning_rate=0.01, epochs=2000, regularization='l2', reg_lambda=0.1)
lin_reg_scratch.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_scratch = lin_reg_scratch.predict(X_reg_test_scaled)

print("Метрики для классификации (собственная реализация):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_scratch):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_scratch, average='weighted'):.2f}")

print("\nМетрики для регрессии (собственная реализация):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_scratch):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_scratch):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_scratch):.2f}")


Метрики для классификации (собственная реализация):
Accuracy: 0.88
F1-Score: 0.83

Метрики для регрессии (собственная реализация):
MSE: 6.57
MAE: 2.07
R² Score: 0.83


## Лабораторная работа №3 (Проведение исследований с решающим деревом) 

**3a. Формулирование гипотез**
1) Подбор гиперпараметров

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])  # 0 - Female, 1 - Male
data['BP'] = label_encoder.fit_transform(data['BP'])    # 0 - HIGH, 1 - LOW, 2 - NORMAL
data['Cholesterol'] = label_encoder.fit_transform(data['Cholesterol'])  # 0 - HIGH, 1 - NORMAL
data['Drug'] = label_encoder.fit_transform(data['Drug'])

X_class = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']]
y_class = data['Drug']
X_reg = data[['Age', 'Sex', 'BP', 'Cholesterol']]
y_reg = data['Na_to_K']

X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_class_train, y_class_train)
y_class_pred = clf.predict(X_class_test)

reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_reg_train, y_reg_train)
y_reg_pred = reg.predict(X_reg_test)

print("Метрики для классификации (бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.2f}")

print("\nМетрики для регрессии (бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred):.2f}")

param_grid_class = {'max_depth': [3, 5, 10, None], 'min_samples_split': [2, 5, 10]}
grid_search_class = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_class, scoring='f1_weighted', cv=5)
grid_search_class.fit(X_class_train, y_class_train)

param_grid_reg = {'max_depth': [3, 5, 10, None], 'min_samples_split': [2, 5, 10]}
grid_search_reg = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_reg, scoring='neg_mean_squared_error', cv=5)
grid_search_reg.fit(X_reg_train, y_reg_train)

print(f"Лучшие параметры для классификации: {grid_search_class.best_params_}")
print(f"Лучшие параметры для регрессии: {grid_search_reg.best_params_}")

clf_best = grid_search_class.best_estimator_
clf_best.fit(X_class_train, y_class_train)
y_class_pred_best = clf_best.predict(X_class_test)

reg_best = grid_search_reg.best_estimator_
reg_best.fit(X_reg_train, y_reg_train)
y_reg_pred_best = reg_best.predict(X_reg_test)

print("\nМетрики для классификации (улучшенный бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_best):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_best, average='weighted'):.2f}")

print("\nМетрики для регрессии (улучшенный бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_best):.2f}")


Метрики для классификации (бейзлайн):
Accuracy: 0.96
F1-Score: 0.96

Метрики для регрессии (бейзлайн):
MSE: 120.64
MAE: 8.30
R² Score: -1.40
Лучшие параметры для классификации: {'max_depth': 5, 'min_samples_split': 2}
Лучшие параметры для регрессии: {'max_depth': 3, 'min_samples_split': 10}

Метрики для классификации (улучшенный бейзлайн):
Accuracy: 0.97
F1-Score: 0.96

Метрики для регрессии (улучшенный бейзлайн):
MSE: 60.17
MAE: 6.56
R² Score: -0.20


### Имплементация алгоритма машинного обучения 

In [96]:
class DecisionTreeFromScratch:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def predict(self, X):
        return np.array([self._predict_sample(sample, self.tree) for sample in X])

    def _build_tree(self, X, y, depth):
        if len(y) < self.min_samples_split or (self.max_depth is not None and depth >= self.max_depth):
            return self._create_leaf(y)

        best_split = self._find_best_split(X, y)
        if not best_split:
            return self._create_leaf(y)

        left_tree = self._build_tree(X[best_split['indices_left']], y[best_split['indices_left']], depth + 1)
        right_tree = self._build_tree(X[best_split['indices_right']], y[best_split['indices_right']], depth + 1)
        return {
            'feature_index': best_split['feature_index'],
            'threshold': best_split['threshold'],
            'left': left_tree,
            'right': right_tree
        }

    def _find_best_split(self, X, y):
        best_split = None
        best_score = float('inf') 

        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                indices_left = X[:, feature_index] <= threshold
                indices_right = X[:, feature_index] > threshold
                if len(indices_left) == 0 or len(indices_right) == 0:
                    continue

                score = self._calculate_split_score(y[indices_left], y[indices_right])
                if score < best_score:
                    best_score = score
                    best_split = {
                        'feature_index': feature_index,
                        'threshold': threshold,
                        'indices_left': indices_left,
                        'indices_right': indices_right
                    }
        return best_split

    def _calculate_split_score(self, y_left, y_right):
        if len(y_left) == 0 or len(y_right) == 0:
            return float('inf')
        
        if np.issubdtype(y_left.dtype, np.integer):
            return (len(y_left) * self._gini(y_left) + len(y_right) * self._gini(y_right)) / (len(y_left) + len(y_right))
        else:
            return (len(y_left) * self._mse(y_left) + len(y_right) * self._mse(y_right)) / (len(y_left) + len(y_right))

    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities**2)

    def _mse(self, y):
        mean = np.mean(y)
        return np.mean((y - mean)**2)

    def _create_leaf(self, y):
        if np.issubdtype(y.dtype, np.integer):
            return np.bincount(y).argmax()
        else:
            return np.mean(y)

    def _predict_sample(self, sample, tree):
        if isinstance(tree, dict):
            feature_value = sample[tree['feature_index']]
            if feature_value <= tree['threshold']:
                return self._predict_sample(sample, tree['left'])
            else:
                return self._predict_sample(sample, tree['right'])
        else:
            return tree

clf = DecisionTreeFromScratch(max_depth=5, min_samples_split=2)
clf.fit(X_class_train.values, y_class_train.values)
y_pred = clf.predict(X_class_test.values)

print("Accuracy:", accuracy_score(y_class_test, y_pred))
print("F1-Score:", f1_score(y_class_test, y_pred, average='weighted'))

reg = DecisionTreeFromScratch(max_depth=5, min_samples_split=2)
reg.fit(X_reg_train.values, y_reg_train.values)
y_pred = reg.predict(X_reg_test.values)

print("MSE:", mean_squared_error(y_reg_test, y_pred))
print("MAE:", mean_absolute_error(y_reg_test, y_pred))
print("R² Score:", r2_score(y_reg_test, y_pred))


Accuracy: 0.98
F1-Score: 0.98
MSE: 12.42107825428839
MAE: 2.4547285606060614
R² Score: 0.7524326215882193


## Лабораторная работа №4 (Проведение исследований со случайным лесом) 

**3a. Формулирование гипотез**
1) Подбор гиперпараметров

In [107]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

clf = RandomForestClassifier(random_state=42)
clf.fit(X_class_train, y_class_train)
y_class_pred = clf.predict(X_class_test)

reg = RandomForestRegressor(random_state=42)
reg.fit(X_reg_train, y_reg_train)
y_reg_pred = reg.predict(X_reg_test)

print("Метрики для классификации (бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.2f}")

print("\nМетрики для регрессии (бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred):.2f}")

param_grid_class = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}
param_grid_reg = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

grid_search_class = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_class, scoring='f1_weighted', cv=5)
grid_search_class.fit(X_class_train, y_class_train)

grid_search_reg = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_reg, scoring='neg_mean_squared_error', cv=5)
grid_search_reg.fit(X_reg_train, y_reg_train)

print(f"Лучшие параметры для классификации: {grid_search_class.best_params_}")
print(f"Лучшие параметры для регрессии: {grid_search_reg.best_params_}")

clf_best = grid_search_class.best_estimator_
clf_best.fit(X_class_train, y_class_train)
y_class_pred_best = clf_best.predict(X_class_test)

reg_best = grid_search_reg.best_estimator_
reg_best.fit(X_reg_train, y_reg_train)
y_reg_pred_best = reg_best.predict(X_reg_test)

print("\nМетрики для классификации (улучшенный бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_best):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_best, average='weighted'):.2f}")

print("\nМетрики для регрессии (улучшенный бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_best):.2f}")


Метрики для классификации (бейзлайн):
Accuracy: 0.89
F1-Score: 0.88

Метрики для регрессии (бейзлайн):
MSE: 71.92
MAE: 7.02
R² Score: -0.43
Лучшие параметры для классификации: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200}
Лучшие параметры для регрессии: {'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 100}

Метрики для классификации (улучшенный бейзлайн):
Accuracy: 0.92
F1-Score: 0.92

Метрики для регрессии (улучшенный бейзлайн):
MSE: 55.74
MAE: 6.40
R² Score: -0.11


### Имплементация алгоритма машинного обучения 

In [111]:
from sklearn.utils import resample

class RandomForestFromScratchClassifier:
    def __init__(self, n_estimators=10, max_depth=None, max_features="sqrt", random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        n_samples, n_features = X.shape

        for _ in range(self.n_estimators):
            X_sample, y_sample = resample(X, y, random_state=self.random_state)

            if self.max_features == "sqrt":
                n_sub_features = int(np.sqrt(n_features))
            elif self.max_features == "log2":
                n_sub_features = int(np.log2(n_features))
            else:
                n_sub_features = n_features

            feature_indices = np.random.choice(n_features, n_sub_features, replace=False)

            tree = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X_sample[:, feature_indices], y_sample)
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        predictions = np.zeros((len(X), len(self.trees)))

        for i, (tree, feature_indices) in enumerate(self.trees):
            predictions[:, i] = tree.predict(X[:, feature_indices])
        return np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)

class RandomForestFromScratchRegressor:
    def __init__(self, n_estimators=10, max_depth=None, max_features="sqrt", random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        n_samples, n_features = X.shape

        for _ in range(self.n_estimators):
            X_sample, y_sample = resample(X, y, random_state=self.random_state)
            if self.max_features == "sqrt":
                n_sub_features = int(np.sqrt(n_features))
            elif self.max_features == "log2":
                n_sub_features = int(np.log2(n_features))
            else:
                n_sub_features = n_features

            feature_indices = np.random.choice(n_features, n_sub_features, replace=False)
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X_sample[:, feature_indices], y_sample)
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        predictions = np.zeros((len(X), len(self.trees)))

        for i, (tree, feature_indices) in enumerate(self.trees):
            predictions[:, i] = tree.predict(X[:, feature_indices])
        return np.mean(predictions, axis=1)


rf_classifier = RandomForestFromScratchClassifier(n_estimators=10, max_depth=5, random_state=42)
rf_classifier.fit(X_class_train.values, y_class_train.values)
y_class_pred = rf_classifier.predict(X_class_test.values)

print("Метрики для классификации (собственная реализация):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.2f}")

rf_regressor = RandomForestFromScratchRegressor(n_estimators=10, max_depth=5, random_state=42)
rf_regressor.fit(X_reg_train.values, y_reg_train.values)
y_reg_pred = rf_regressor.predict(X_reg_test.values)

print("\nМетрики для регрессии (собственная реализация):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred):.2f}")


Метрики для классификации (собственная реализация):
Accuracy: 0.68
F1-Score: 0.57

Метрики для регрессии (собственная реализация):
MSE: 54.73
MAE: 6.31
R² Score: -0.09


## Лабораторная работа №5 (Проведение исследований с градиентным бустингом) 

**3a. Формулирование гипотез**
1) Подбор гиперпараметров

In [112]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

clf = GradientBoostingClassifier(random_state=42)
clf.fit(X_class_train_scaled, y_class_train)
y_class_pred = clf.predict(X_class_test_scaled)

reg = GradientBoostingRegressor(random_state=42)
reg.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred = reg.predict(X_reg_test_scaled)

print("Метрики для классификации (бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.2f}")

print("\nМетрики для регрессии (бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred):.2f}")

param_grid_class = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}
param_grid_reg = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search_class = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid_class,
    scoring='f1_weighted',
    cv=5
)
grid_search_class.fit(X_class_train_scaled, y_class_train)

grid_search_reg = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid_reg,
    scoring='neg_mean_squared_error',
    cv=5
)
grid_search_reg.fit(X_reg_train_scaled, y_reg_train)

print(f"Лучшие параметры для классификации: {grid_search_class.best_params_}")
print(f"Лучшие параметры для регрессии: {grid_search_reg.best_params_}")

clf_best = grid_search_class.best_estimator_
clf_best.fit(X_class_train_scaled, y_class_train)
y_class_pred_best = clf_best.predict(X_class_test_scaled)

reg_best = grid_search_reg.best_estimator_
reg_best.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_best = reg_best.predict(X_reg_test_scaled)

print("\nМетрики для классификации (улучшенный бейзлайн):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_best):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_best, average='weighted'):.2f}")

print("\nМетрики для регрессии (улучшенный бейзлайн):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_best):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_best):.2f}")

Метрики для классификации (бейзлайн):
Accuracy: 0.90
F1-Score: 0.89

Метрики для регрессии (бейзлайн):
MSE: 70.63
MAE: 6.78
R² Score: -0.41
Лучшие параметры для классификации: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Лучшие параметры для регрессии: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}

Метрики для классификации (улучшенный бейзлайн):
Accuracy: 0.92
F1-Score: 0.91

Метрики для регрессии (улучшенный бейзлайн):
MSE: 52.78
MAE: 6.25
R² Score: -0.05


### Имплементация алгоритма машинного обучения 

In [113]:
class GradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, task="regression"):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.task = task
        self.models = []

    def _gradient(self, y_true, y_pred):
        if self.task == "regression":
            return y_true - y_pred
        elif self.task == "classification":
            return y_true - 1 / (1 + np.exp(-y_pred))

    def fit(self, X, y):
        self.models = []
        y_pred = np.zeros_like(y, dtype=float)

        for _ in range(self.n_estimators):
            residuals = self._gradient(y, y_pred)
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.models.append(tree)
            y_pred += self.learning_rate * tree.predict(X)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for tree in self.models:
            y_pred += self.learning_rate * tree.predict(X)

        if self.task == "classification":
            return (y_pred > 0).astype(int)
        return y_pred


gb_clf = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3, task="classification")
gb_clf.fit(X_class_train_scaled, y_class_train)
y_class_pred_impl = gb_clf.predict(X_class_test_scaled)

gb_reg = GradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3, task="regression")
gb_reg.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_impl = gb_reg.predict(X_reg_test_scaled)

print("Метрики для классификации (реализация):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_impl):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_impl, average='weighted'):.2f}")

print("\nМетрики для регрессии (реализация):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_impl):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_impl):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_impl):.2f}")

gb_clf_tuned = GradientBoosting(n_estimators=200, learning_rate=0.05, max_depth=5, task="classification")
gb_clf_tuned.fit(X_class_train_scaled, y_class_train)
y_class_pred_tuned = gb_clf_tuned.predict(X_class_test_scaled)

gb_reg_tuned = GradientBoosting(n_estimators=200, learning_rate=0.05, max_depth=5, task="regression")
gb_reg_tuned.fit(X_reg_train_scaled, y_reg_train)
y_reg_pred_tuned = gb_reg_tuned.predict(X_reg_test_scaled)

print("\nМетрики для классификации (улучшенная реализация):")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred_tuned):.2f}")
print(f"F1-Score: {f1_score(y_class_test, y_class_pred_tuned, average='weighted'):.2f}")

print("\nМетрики для регрессии (улучшенная реализация):")
print(f"MSE: {mean_squared_error(y_reg_test, y_reg_pred_tuned):.2f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_tuned):.2f}")
print(f"R² Score: {r2_score(y_reg_test, y_reg_pred_tuned):.2f}")

Метрики для классификации (реализация):
Accuracy: 0.73
F1-Score: 0.63

Метрики для регрессии (реализация):
MSE: 70.56
MAE: 6.75
R² Score: -0.41

Метрики для классификации (улучшенная реализация):
Accuracy: 0.75
F1-Score: 0.64

Метрики для регрессии (улучшенная реализация):
MSE: 64.77
MAE: 6.48
R² Score: -0.29


## Итоги

1) **ЛР1 KNN** (K-Nearest Neighbors):

- `Классификация`: Улучшенный бейзлайн и самостоятельная имплементация показывают одинаковые результаты (0.93), что указывает на стабильность алгоритма
- `Регрессия`: Улучшение бейзлайна с 6.63 до 3.21, но самостоятельная имплементация дала результат хуже улучшенного бейзлайна (3.60)

2) **ЛР2 Линейные модели:**

- `Классификация`: Улучшенный бейзлайн (0.97) значительно превосходит самостоятельную имплементацию (0.88)
- `Регрессия`: Все результаты близки (различия незначительные), что указывает на стабильность метода

3) **ЛР3 Решающее дерево:**

- `Классификация`: Результаты показывают улучшение в самостоятельной имплементации (0.98 против 0.96 для бейзлайна)
- `Регрессия`: Существенное улучшение с 8.30 до 2.46 в самостоятельной имплементации, что подтверждает эффективность алгоритма

4) **ЛР4 Случайный лес:**

- `Классификация`: Самостоятельная имплементация дала наименьший результат (0.68), что указывает на неудачную настройку
- `Регрессия`: Улучшение с 7.02 до 6.40, и самостоятельная имплементация не привела к значительным улучшениям (6.31) - удивительно

5) **ЛР5 Градиентный бустинг:**

- `Классификация`: Результаты улучшились с 0.90 до 0.92 в улучшенном бейзлайне, но самостоятельная имплементация дала хуже (0.75)
- `Регрессия`: Улучшение с 6.78 до 6.25, но самостоятельная имплементация оказалась незначительно хуже (6.75)

### Сравнение

- `Лучшие результаты **по классификации**` продемонстрировали `решающее дерево (0.98)` и `линейные модели (0.97)`
- `Лучшие результаты **по регрессии**` показало `решающее дерево (2.46)`
- Самостоятельные имплементации в большинстве случаев дают более слабые результаты, чем улучшенные бейзлайны, особенно для случайного леса и градиентного бустинга (там получился вообще какой-то ужас)